<a href="https://www.kaggle.com/code/martinvelkov/digit-recognizer?scriptVersionId=140394892" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from pandas import read_csv, DataFrame
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical, normalize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
train = read_csv('/kaggle/input/digit-recognizer/train.csv')
test = read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
X_train = train.drop(['label'], axis=1).values.astype('float32')
y_train = train['label'].values.astype('int32')
X_test = test.values.astype('float32')

In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [5]:
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [6]:
y_train = to_categorical(y_train, num_classes=10)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train, shuffle=True)

In [8]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

In [9]:
datagen.fit(X_train)

In [10]:
model = Sequential([
    Conv2D(32, (5, 5), padding='same', activation=relu, input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(32, (5, 5), padding='same', activation=relu),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding='same', activation=relu),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation=relu),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation=relu),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation=softmax)
])

In [11]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [13]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=0.00001)

In [14]:
model.fit(datagen.flow(X_train, y_train, batch_size=64), epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/100


2023-08-19 16:08:37.101906: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


591/591 [==============================] - 21s 24ms/step - loss: 0.3487 - accuracy: 0.8933 - val_loss: 0.5004 - val_accuracy: 0.8295 - lr: 0.0010
Epoch 2/100
591/591 [==============================] - 14s 24ms/step - loss: 0.1269 - accuracy: 0.9606 - val_loss: 0.0564 - val_accuracy: 0.9840 - lr: 0.0010
Epoch 3/100
591/591 [==============================] - 13s 22ms/step - loss: 0.0964 - accuracy: 0.9706 - val_loss: 0.0464 - val_accuracy: 0.9867 - lr: 0.0010
Epoch 4/100
591/591 [==============================] - 14s 24ms/step - loss: 0.0802 - accuracy: 0.9754 - val_loss: 0.0606 - val_accuracy: 0.9819 - lr: 0.0010
Epoch 5/100
591/591 [==============================] - 13s 23ms/step - loss: 0.0704 - accuracy: 0.9777 - val_loss: 0.0404 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 6/100
591/591 [==============================] - 14s 23ms/step - loss: 0.0670 - accuracy: 0.9791 - val_loss: 0.0331 - val_accuracy: 0.9907 - lr: 0.0010
Epoch 7/100
591/591 [==============================] - 14s 24ms/

In [15]:
y_pred = model.predict(X_test)

875/875 [==============================] - 2s 2ms/step


In [16]:
submission = DataFrame({
    'ImageId': range(1, len(y_pred)+1),
    'Label': y_pred.argmax(axis=1)
}) 

In [17]:
submission.to_csv('/kaggle/working/submission.csv', index=False)